In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
from keras.layers import Input
from PIL import Image
from matplotlib import pyplot as plt
import cv2 
import numpy as np
from model import Deeplabv3
import time

Using TensorFlow backend.


In [3]:
# load model
deeplab_model = Deeplabv3(weights='pascal_voc', backbone='xception',input_shape=(512,512,3), OS=8)

W0917 10:36:57.140460 4526147008 deprecation.py:506] From /Users/ngao024/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
trained_image_width=512 
mean_subtraction_value=127.5
img = np.array(Image.open('test_images/IMG_9181.jpg'))

w, h, _ = img.shape
ratio = float(trained_image_width) / np.max([w, h])
resized_img = cv2.resize(img, (int(ratio*h), int(ratio*w)))

# apply normalization for trained dataset images
resized_img = (resized_img / mean_subtraction_value) - 1.

pad_x = int(trained_image_width - resized_img.shape[0])
pad_y = int(trained_image_width - resized_img.shape[1])

resized_img = np.pad(resized_img, ((0, pad_x), (0, pad_y), (0, 0)), mode='constant')

start = time.time()
res = new_deeplab_model.predict(np.expand_dims(resized_img, 0))
labels = np.argmax(res.squeeze(),-1)
print(time.time() - start, ' seconds elapse..')

if pad_x > 0:
    labels = labels[:-pad_x]
if pad_y > 0:
    labels = labels[:, :-pad_y]
labels = np.array(Image.fromarray(labels.astype('uint8')).resize((h, w)))

plt.figure(figsize=(8,6))
plt.imshow(img)
plt.imshow(labels, alpha=0.4)
plt.show()